In [24]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')


Libraries imported.


### Part 1: Parsing html data

In [25]:
tab = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#read_html returns a list so we parse through with list indexing
tab = tab[0]
#rename the postal code column forconsistency
tab.rename(columns={"Postal Code":"PostalCode"},inplace=True)
tab.head(15)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [26]:
#drop rows/entries with 'Not assigned' values and reset the index
tab = tab[tab.Borough != 'Not assigned']
tab.reset_index(inplace=True)
tab.drop('index',axis=1,inplace=True)
tab.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### To find out the number of rows contained in our data we use the .shape method

In [27]:
print("Our Dataframe has {} rows".format(tab.shape[0]))

Our Dataframe has 103 rows


## Part 2: Getting the location coordinates

In [28]:
!pip install geocoder

In [29]:
#read in the csv file since geocode API was not responsive
cordtable = pd.read_csv('https://cocl.us/Geospatial_data')
cordtable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
#rename the Postal Code column for consistency
cordtable.rename(index=str, columns={'Postal Code': 'PostalCode'}, inplace=True)
cordtable.head(3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [31]:
#Here i perform a merge joining the new dataframe to the previous one
cleantable = pd.merge(tab,cordtable,how='outer', on=['PostalCode'])
cleantable.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [32]:
cleantable.shape
#Now we have 5 columns with the addition of the Lat and Long Coordinates

(103, 5)

## Part 3: Exploring Toronto

In [33]:
#selecting rows with Boroughs that contain "Toronto"
Toronto = cleantable[cleantable['Borough'].str.contains("Toronto")] 
#reset the index 
Toronto.reset_index(drop=True,inplace=True)
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [0]:
import folium #library for mapping 
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim 

In [35]:
#use the geocode API to get the longitude and latitude of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Creating map with markers indicating the Neighbourhoods**

In [0]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

In [37]:
map_Toronto

**Exploring Neighbourhoods with Foursquare**

In [0]:
CLIENT_ID = 'FY3KY3L1X23ZAIWOYIGBYVTOEY0E2TJ53YWR0UTONDSU3SOR' 
CLIENT_SECRET = 'CPKKORINNFNOUKCGLUQV1WWFU5A2GGDS5LYBIWVTOJN3ULR0' 
VERSION = '20180605' 
LIMIT = 20
radius = 500

In [39]:
neighborhood_latitude = Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [40]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=FY3KY3L1X23ZAIWOYIGBYVTOEY0E2TJ53YWR0UTONDSU3SOR&client_secret=CPKKORINNFNOUKCGLUQV1WWFU5A2GGDS5LYBIWVTOJN3ULR0&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=20'

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edea585963d29001b5876d5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


**We setup a function to explore the neighborhoods in the Dataframe**

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [49]:
print(Toronto_venues.shape)
Toronto_venues.head()

(641, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


**Checking for the number of unique Neighborhoods associated with our venues**

In [50]:
Toronto_venues['Neighborhood'].value_counts()

Little Portugal, Trinity                                                                                      20
University of Toronto, Harbord                                                                                20
Central Bay Street                                                                                            20
The Annex, North Midtown, Yorkville                                                                           20
North Toronto West, Lawrence Park                                                                             20
Church and Wellesley                                                                                          20
Stn A PO Boxes                                                                                                20
Runnymede, Swansea                                                                                            20
The Danforth West, Riverdale                                                                    

**find out how many unique categories can be curated from all the returned venues**

In [51]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 171 uniques categories.


In [0]:
#sets the display property to make all columns visible,helpful in slicing and rearranging your columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Analyze each Neighborhood


In [54]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot.rename(columns={'Neighborhood':'Neighborhoodx'},inplace=True)
#adjusting the Dataframe so it starts with the Neighbourhood column
neighborhood_toronto = Toronto_venues[['Neighborhood']]
clean_toronto = pd.concat([neighborhood_toronto,Toronto_onehot],axis =1)
clean_toronto.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhoodx,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [56]:
#Checkout the shape of the new dataframe
clean_toronto.shape

(641, 172)

In [55]:
toronto_grouped = clean_toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhoodx,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.0000,0.0000,0.050,0.0000,0.000000,0.05,0.05,0.00,0.00,0.05,0.0000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.00,0.0000,0.00,0.05,0.00,0.00,0.00,0.00,0.05,0.050000,0.00,0.00,0.0000,0.05,0.00,0.00,0.000000,0.00,0.00,0.000,0.000000,0.0000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.0500,0.000000,0.000000,0.00,0.00,0.000,0.00,0.05,0.05,0.0000,0.00,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.0000,0.000000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.050000,0.00,0.00,0.00,0.0000,0.00,0.000000,0.05,0.0000,0.00,0.00,0.0000,0.00,0.00,0.050000,0.00,0.000000,0.000,0.000000,0.0000,0.00,0.00,0.00,0.0000,0.000000,0.00,0.0000,0.050000,0.00,0.00,0.000000,0.0000,0.10,0.00,0.0000,0.00,0.00,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,0.000000,0.00,0.0000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.0000,0.00,0.00,0.0000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.00,0.00,0.00,0.00,0.0000,0.00,0.0000,0.0000,0.050,0.0000,0.050000,0.00,0.00,0.00,0.00,0.00,0.0000,0.000000,0.100000,0.000000,0.00,0.00,0.050000,0.000000,0.00,0.1500,0.00,0.0000,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.100000,0.00,0.00,0.0000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000,0.000000,0.0000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.0000,0.000000,0.000000,0.00,0.00,0.000,0.00,0.00,0.00,0.0000,0.00,0.0500,0.0000,0.0000,0.00,0

**Top 5 Venues in each Neighborhood**

In [57]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1          Restaurant  0.05
2  Basketball Stadium  0.05
3        Concert Hall  0.05
4                Park  0.05


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.15
1             Coffee Shop  0.10
2          Breakfast Spot  0.10
3                 Stadium  0.05
4  Furniture / Home Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1               Brewery  0.06
2                   Spa  0.06
3            Skate Park  0.06
4            Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4     Boat or Ferry  0.06


----Central Bay

In [0]:
#Casting the data into a Dataframe using a function,let's define the function first
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Bistro,Bakery,Park,Coffee Shop,Cocktail Bar,Museum,Cheese Shop,Restaurant,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Burrito Place,Restaurant,Bar,Italian Restaurant,Bakery,Climbing Gym,Stadium
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Spa,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Sculpture Garden,Coffee Shop,Plane,Rental Car Location,Boat or Ferry,Airport Food Court
4,Central Bay Street,Coffee Shop,Spa,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Japanese Restaurant


# Clustering the Neighborhood


In [61]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 4, 1, 4, 4, 4, 1, 1], dtype=int32)

In [75]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.drop(['PostalCode','Borough'],axis=1,inplace=True)
toronto_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Bakery,Chocolate Shop,Pub,Restaurant,Dessert Shop,Distribution Center,Spa
1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Creperie,Diner,Mexican Restaurant,Discount Store,Burrito Place,Distribution Center
2,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Electronics Store,Steakhouse,Ramen Restaurant,Sporting Goods Shop,Shopping Mall,Plaza,Mexican Restaurant,Tea Room,Thai Restaurant
3,St. James Town,43.651494,-79.375418,4,Gastropub,Coffee Shop,Restaurant,BBQ Joint,Poke Place,Food Truck,Café,Middle Eastern Restaurant,Hotel,Theater
4,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhoodx,Yoga Studio,Department Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


**Visualizing the Clusters on the map**

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Finding and Examining Clusters**

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(3, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,0,Health Food Store,Trail,Pub,Neighborhoodx,Yoga Studio,Department Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center


In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(3, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Park,Breakfast Spot,Bakery,Chocolate Shop,Pub,Restaurant,Dessert Shop,Distribution Center,Spa
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Creperie,Diner,Mexican Restaurant,Discount Store,Burrito Place,Distribution Center
5,Berczy Park,1,Seafood Restaurant,Bistro,Bakery,Park,Coffee Shop,Cocktail Bar,Museum,Cheese Shop,Restaurant,Liquor Store
6,Central Bay Street,1,Coffee Shop,Spa,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Japanese Restaurant
11,"Little Portugal, Trinity",1,Bar,Yoga Studio,Asian Restaurant,Greek Restaurant,French Restaurant,Ice Cream Shop,Japanese Restaurant,Korean Restaurant,Cuban Restaurant,New American Restaurant
12,"The Danforth West, Riverdale",1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Bookstore,Fruit & Vegetable Store,Juice Bar,Dessert Shop,Cosmetics Shop,Pizza Place,Pub
15,"India Bazaar, The Beaches West",1,Fast Food Restaurant,Ice Cream Shop,Pub,Burrito Place,Sandwich Place,Liquor Store,Restaurant,Italian Restaurant,Intersection,Steakhouse
20,Davisville North,1,Gym / Fitness Center,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Sandwich Place,Pizza Place,Park,Creperie,Cosmetics Shop
25,"Parkdale, Roncesvalles",1,Breakfast Spot,Gift Shop,Restaurant,Eastern European Restaurant,Dog Run,Bar,Movie Theater,Italian Restaurant,Bookstore,Dessert Shop
26,Davisville,1,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Pizza Place,Café,Diner,Sandwich Place,Seafood Restaurant,Brewery


In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(3, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,2,Bus Line,Park,Swim School,Yoga Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
21,"Forest Hill North & West, Forest Hill Road Park",2,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center
33,Rosedale,2,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(3, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,3,Garden,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(3, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",4,Café,Electronics Store,Steakhouse,Ramen Restaurant,Sporting Goods Shop,Shopping Mall,Plaza,Mexican Restaurant,Tea Room,Thai Restaurant
3,St. James Town,4,Gastropub,Coffee Shop,Restaurant,BBQ Joint,Poke Place,Food Truck,Café,Middle Eastern Restaurant,Hotel,Theater
7,Christie,4,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Nightclub,Restaurant,Candy Store,Coffee Shop
8,"Richmond, Adelaide, King",4,Coffee Shop,Gym / Fitness Center,Sushi Restaurant,Hotel,Lounge,Concert Hall,Neighborhoodx,Opera House,Pizza Place,Plaza
9,"Dufferin, Dovercourt Village",4,Pharmacy,Bakery,Music Venue,Portuguese Restaurant,Bank,Supermarket,Middle Eastern Restaurant,Brewery,Café,Bar
10,"Harbourfront East, Union Station, Toronto Islands",4,Park,Hotel,Plaza,Lake,Bubble Tea Shop,Sporting Goods Shop,Dessert Shop,Japanese Restaurant,Salad Place,Supermarket
13,"Toronto Dominion Centre, Design Exchange",4,Coffee Shop,Café,Deli / Bodega,Tea Room,Pub,Restaurant,Sandwich Place,Beer Bar,Japanese Restaurant,Steakhouse
14,"Brockton, Parkdale Village, Exhibition Place",4,Café,Coffee Shop,Breakfast Spot,Burrito Place,Restaurant,Bar,Italian Restaurant,Bakery,Climbing Gym,Stadium
16,"Commerce Court, Victoria Hotel",4,Café,Coffee Shop,Gym / Fitness Center,Museum,Bookstore,Japanese Restaurant,Restaurant,Bakery,Pub,Tea Room
17,Studio District,4,Coffee Shop,Café,Comfort Food Restaurant,Stationery Store,Italian Restaurant,Middle Eastern Restaurant,Bakery,Ice Cream Shop,Cheese Shop,Sandwich Place
